# Try word2vec on song playlists

In [18]:
import os
import csv
import gensim, logging

import time

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [19]:
playlist_dir = "C:/Users/Colin/Documents/Playlist maker/spotify_playlists/"
#playlist_dir = "C:/Users/Colin/Documents/Playlist maker/demo_playlists/"

In [51]:
#Create playlist iterator object
#__iter__ functions like a 'next'

#iterate though all rows of all csv files
#needs to yield sentences (playlists)

class MyPlaylists(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        #fnames = [os.listdir(self.dirname)[0]]
        fnames = os.listdir(self.dirname)
        for fname in fnames: #just first
            print('Starting playlist: ' + fname)
            fullname = self.dirname + fname
            
            with open(fullname, 'rt', encoding='utf-8') as csvfile: 
                
                csvfile.seek(5)
                print(csvfile.readline())
                dialect = csv.Sniffer().sniff(csvfile.readline(), delimiters=';|')
                print(dialect.delimiter)
                time.sleep(1)

                csvfile.seek(0)
                spamReader = csv.reader(csvfile, delimiter=dialect.delimiter)
                next(spamReader) #skip first playlist title
                i = 0
                title = False

                playlist = []
                #row [name, artist, id, sample-url]
                for row in spamReader:
                    #print(row)
                    #time.sleep(0.2)

                    i+= 1
                    #print(row.split(";"))
                    try:
                        #print(row[0])
                        if row[0] == '':
                            #print('empty row')
                            if i % 1000 == 0:
                                print('Reached playlist: ' + str(i))
                            yield playlist
                            title = True
                        elif title == True:
                            title = False  
                        else:
                            artist = row[0].replace(" ", "-").lower()
                            song = row[1].replace(" ", "-").lower()
                            playlist.append(artist + "__" + song)
                    except IndexError as e:
                        print(e)
                        print('Index error at row: ' + str(i))
                        pass

                #return final playlist
                yield playlist

playlists = MyPlaylists(playlist_dir) # a memory-friendly iterator

In [52]:
#Test iterator
#for play in playlists:
#    print(play)

# Set up model

In [53]:
w2v_model = gensim.models.Word2Vec(min_count=10, workers=2)

In [ ]:
t = time.time()

w2v_model.build_vocab(playlists, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time.time() - t) / 60, 2)))

INFO - 19:50:29: collecting all words and their counts


Starting playlist: playlists_a.csv
 AA Antiguas/Viejas 👹

|


INFO - 19:50:30: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Starting playlist: playlists_b.csv
a Americana;;;;;

;
Reached playlist: 84000
Starting playlist: playlists_c.csv
ngue en casa;;;;;

;
Reached playlist: 4000
Reached playlist: 10000
Reached playlist: 98000
Reached playlist: 180000
Starting playlist: playlists_d.csv
 Brandneu

|
list index out of range
Index error at row: 27300
list index out of range
Index error at row: 27592
Reached playlist: 122000
Starting playlist: playlists_e.csv
 & Easy

|
list index out of range
Index error at row: 114703


INFO - 19:56:47: PROGRESS: at sentence #10000, processed 725885823 words, keeping 292489 word types


Starting playlist: playlists_f.csv
avorites

|
list index out of range
Index error at row: 4060
Reached playlist: 54000
Reached playlist: 85000
Reached playlist: 89000
Reached playlist: 125000
Reached playlist: 128000
Reached playlist: 166000


In [ ]:
#Number of sentences
w2v_model.corpus_count

In [ ]:
#Vocab length
len(w2v_model.wv.vocab)

In [ ]:
t = time.time()

w2v_model.train(playlists, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)

print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
w2v_model['climax__air-please']

In [ ]:
w2v_model['aretha-franklin__respect']

In [ ]:
len(w2v_model.wv.vocab)


In [ ]:
w2v_model.most_similar(positive=['aretha-franklin__respect'])

In [ ]:
w2v_model.most_similar(positive=['tina-turner__the-best'])

In [ ]:
w2v_model.wv.similarity('tina-turner__the-best', 'aretha-franklin__respect')

In [ ]:
w2v_model.wv.similarity('men-at-work__down-under', 'toto__africa')

In [ ]:
w2v_model.most_similar(positive=['toto__africa'])